In [1]:
# Dependencies
import numpy as np
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [2]:
engine = create_engine("sqlite:///hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [3]:
# Create our session (link) from Python to the DB
session = Session(engine)

## D1: Determine the Summary Statistics for June

In [4]:
# 1. Import the sqlalchemy extract function.
from sqlalchemy import extract

# 2. Write a query that filters the Measurement table to retrieve the temperatures for the month of June. 
June_Temperature = session.query(Measurement).filter(extract('month', Measurement.date) == 6)
print(June_Temperature)

SELECT measurement.id AS measurement_id, measurement.station AS measurement_station, measurement.date AS measurement_date, measurement.prcp AS measurement_prcp, measurement.tobs AS measurement_tobs 
FROM measurement 
WHERE CAST(STRFTIME('%m', measurement.date) AS INTEGER) = ?


In [5]:
#  3. Convert the June temperatures to a list.
June_Temperature_List = [i.tobs for i in June_Temperature]
print(June_Temperature_List[0:10])


[78.0, 76.0, 78.0, 76.0, 77.0, 78.0, 77.0, 78.0, 78.0, 79.0]


In [6]:
# 4. Create a DataFrame from the list of temperatures for the month of June. 
June_Temperature_df = pd.DataFrame(June_Temperature_List, columns=['June Temperature'])
print(June_Temperature_df.head())



   June Temperature
0              78.0
1              76.0
2              78.0
3              76.0
4              77.0


In [7]:
# 5. Calculate and print out the summary statistics for the June temperature DataFrame.
print(June_Temperature_df.describe())


       June Temperature
count       1700.000000
mean          74.944118
std            3.257417
min           64.000000
25%           73.000000
50%           75.000000
75%           77.000000
max           85.000000


## D2: Determine the Summary Statistics for December

In [8]:
# 6. Write a query that filters the Measurement table to retrieve the temperatures for the month of December.
Dec_Temperature = session.query(Measurement).filter(extract('month', Measurement.date) == 12)
print(Dec_Temperature)


SELECT measurement.id AS measurement_id, measurement.station AS measurement_station, measurement.date AS measurement_date, measurement.prcp AS measurement_prcp, measurement.tobs AS measurement_tobs 
FROM measurement 
WHERE CAST(STRFTIME('%m', measurement.date) AS INTEGER) = ?


In [9]:
# 7. Convert the December temperatures to a list.
Dec_Temperature_List = [i.tobs for i in Dec_Temperature]
print(Dec_Temperature_List[0:10])


[76.0, 74.0, 74.0, 64.0, 64.0, 67.0, 77.0, 66.0, 69.0, 68.0]


In [10]:
# 8. Create a DataFrame from the list of temperatures for the month of December. 
Dec_Temperature_df = pd.DataFrame(Dec_Temperature_List, columns=['December Temperature'])
print(Dec_Temperature_df.head())


   December Temperature
0                  76.0
1                  74.0
2                  74.0
3                  64.0
4                  64.0


In [11]:
# 9. Calculate and print out the summary statistics for the Decemeber temperature DataFrame.
print(Dec_Temperature_df.describe())
print(June_Temperature_df.describe())


       December Temperature
count           1517.000000
mean              71.041529
std                3.745920
min               56.000000
25%               69.000000
50%               71.000000
75%               74.000000
max               83.000000
       June Temperature
count       1700.000000
mean          74.944118
std            3.257417
min           64.000000
25%           73.000000
50%           75.000000
75%           77.000000
max           85.000000


In [23]:
precipitation_in_june = session.query(Measurement.date, Measurement.prcp).filter(extract('month', Measurement.date) == 6).all()
precipitation_in_june_list = [i.prcp for i in precipitation_in_june]
precipitation_in_june_df = pd.DataFrame(precipitation_in_june_list, columns=['June Precipitation'])
precipitation_in_december = session.query(Measurement.date, Measurement.prcp).filter(extract('month', Measurement.date) == 12).all()
precipitation_in_december_list = [i.prcp for i in precipitation_in_december]
precipitation_in_december_df = pd.DataFrame(precipitation_in_december_list, columns=['December Precipitation'])
print(precipitation_in_december_df.describe())
print(precipitation_in_june_df.describe())

       December Precipitation
count             1405.000000
mean                 0.216819
std                  0.541399
min                  0.000000
25%                  0.000000
50%                  0.030000
75%                  0.150000
max                  6.420000
       June Precipitation
count         1574.000000
mean             0.136360
std              0.335731
min              0.000000
25%              0.000000
50%              0.020000
75%              0.120000
max              4.430000


In [36]:
station_names = session.query(Measurement.station).distinct().all()
station_precipitation_june = session.query(Measurement.station, func.avg(Measurement.prcp)).filter(extract('month', Measurement.date) == 6).group_by(Measurement.station).all()
station_precipitation_june_df = pd.DataFrame(station_precipitation_june, columns=['Station', 'June Precipitation'])
station_precipitation_december = session.query(Measurement.station, func.avg(Measurement.prcp)).filter(extract('month', Measurement.date) == 12).group_by(Measurement.station).all()
station_precipitation_december_df = pd.DataFrame(station_precipitation_december, columns=['Station', 'December Precipitation'])
print(station_precipitation_june_df.sort_values(by=['June Precipitation'], ascending=False))
print(station_precipitation_december_df.sort_values(by=['December Precipitation'], ascending=False))




       Station  June Precipitation
3  USC00516128            0.495748
6  USC00519281            0.151525
1  USC00513117            0.118248
2  USC00514830            0.114192
5  USC00518838            0.094615
4  USC00517948            0.057975
8  USC00519523            0.050044
7  USC00519397            0.022661
0  USC00511918            0.015157
       Station  December Precipitation
5  USC00518838                0.638182
3  USC00516128                0.507005
6  USC00519281                0.244931
1  USC00513117                0.203241
8  USC00519523                0.162010
2  USC00514830                0.154966
4  USC00517948                0.152727
0  USC00511918                0.138146
7  USC00519397                0.075314
